In [2]:
import os
import subprocess
from ipywidgets import widgets, VBox, Button, Output
from tqdm.notebook import tqdm
from IPython.display import display

# Define paths to your atlas files and MNI space file
atlas_paths = {
    'network': '/path/to/network_ROIs.nii',
    'visual': '/path/to/visual_ROIs.nii',
    'motor': '/path/to/motor_ROIs.nii'
}
mni_path = '/path/to/bet_mni.nii'

def mirtk_registration(patient_ids, register_network, register_visual, register_motor):
    for patient_id in patient_ids.split(','):  # Splitting the input string by commas
        patient_dir = f'/path/to/directory/{patient_id.strip()}/dti_outputs'  # .strip() to remove any leading/trailing whitespace
        os.chdir(patient_dir)

        b0_nii_path = 'b0.nii'
        b0_exists = os.path.exists(b0_nii_path)

        total_steps = 3 + sum([register_network, register_visual, register_motor])  # Initial steps + conditional atlas registrations
        progress = tqdm(total=total_steps, desc=f'Total Progress for {patient_id.strip()}')

        # Check if b0.nii already exists; if not, convert b0.mif to b0.nii
        if not b0_exists:
            subprocess.run(['mrconvert', 'b0.mif', b0_nii_path])
            progress.update(1)  # Update progress after conversion
        else:
            progress.update(1)  # Update progress, acknowledging that b0.nii was already present

        # BET extraction
        subprocess.run(['bet', b0_nii_path, 'bet_b0.nii', '-m', '-f', 0.25])
        progress.update(1)  # Update progress after BET extraction

        # MNI registration
        subprocess.run(['mirtk', 'register', '-image', 'bet_b0.nii.gz', mni_path,
                        '-dofout', 'transfomation.txt', '-output', 'r_mni.nii'])
        progress.update(1)  # Update progress after MNI registration

        # Perform atlas registrations based on user input
        for atlas_type, do_register in [('network', register_network), 
                                        ('visual', register_visual), 
                                        ('motor', register_motor)]:
            if do_register:
                atlas_path = atlas_paths[atlas_type]
                output_path = f'r_{atlas_type}_ROI.nii'
                subprocess.run(['mirtk', 'transform-image', atlas_path, output_path,
                                '-target', 'bet_b0.nii.gz', '-dofin', 'transfomation.txt', '-interp', 'NN'])
                progress.update(1)  # Update progress after each atlas registration

        progress.close()  # Close the progress bar when done
        print(f'Registration complete for patient: {patient_id.strip()}')

# Interactive Widgets
patient_id_widget = widgets.Text(value='patient_id1, patient_id2', description='Patient IDs:', disabled=False)
network_widget = widgets.Checkbox(value=False, description='Register Network Areas', disabled=False)
visual_widget = widgets.Checkbox(value=False, description='Register Visual Areas', disabled=False)
motor_widget = widgets.Checkbox(value=False, description='Register Motor Areas', disabled=False)

ui = VBox([patient_id_widget, network_widget, visual_widget, motor_widget])
out = Output()

def on_button_clicked(_):
    with out:
        out.clear_output()
        mirtk_registration(patient_id_widget.value, network_widget.value, visual_widget.value, motor_widget.value)

button = Button(description="Run Registration")
button.on_click(on_button_clicked)

display(ui, button, out)


Button(description='Run Registration', style=ButtonStyle())

Output()